In [155]:
import tensorflow as tf
import os
import scipy.io
import numpy as np

In [212]:
working_dir = os.getcwd()

file_path = os.path.join(working_dir, 'PINNs', 'main', 'Data','AC.mat')

data = scipy.io.loadmat(file_path)

In [213]:
# get data for model
x = tf.convert_to_tensor(data['x'].flatten()[:,None])
usol = (np.real(data['uu'])).T[0,None].T

In [214]:
print(x.shape)
print(usol.shape)

(512, 1)
(512, 1)


In [215]:
print(usol)

[[-1.00000000e+00]
 [-9.92128048e-01]
 [-9.84139541e-01]
 [-9.76038002e-01]
 [-9.67826945e-01]
 [-9.59509876e-01]
 [-9.51090295e-01]
 [-9.42571690e-01]
 [-9.33957541e-01]
 [-9.25251317e-01]
 [-9.16456475e-01]
 [-9.07576457e-01]
 [-8.98614696e-01]
 [-8.89574607e-01]
 [-8.80459592e-01]
 [-8.71273038e-01]
 [-8.62018313e-01]
 [-8.52698769e-01]
 [-8.43317741e-01]
 [-8.33878543e-01]
 [-8.24384471e-01]
 [-8.14838800e-01]
 [-8.05244785e-01]
 [-7.95605659e-01]
 [-7.85924631e-01]
 [-7.76204889e-01]
 [-7.66449596e-01]
 [-7.56661892e-01]
 [-7.46844890e-01]
 [-7.37001678e-01]
 [-7.27135318e-01]
 [-7.17248845e-01]
 [-7.07345267e-01]
 [-6.97427562e-01]
 [-6.87498682e-01]
 [-6.77561546e-01]
 [-6.67619046e-01]
 [-6.57674042e-01]
 [-6.47729365e-01]
 [-6.37787813e-01]
 [-6.27852150e-01]
 [-6.17925111e-01]
 [-6.08009397e-01]
 [-5.98107674e-01]
 [-5.88222575e-01]
 [-5.78356698e-01]
 [-5.68512607e-01]
 [-5.58692831e-01]
 [-5.48899861e-01]
 [-5.39136155e-01]
 [-5.29404131e-01]
 [-5.19706173e-01]
 [-5.1004462

In [240]:
#initializer = tf.keras.initializers.GlorotUniform(seed=12)

model = tf.keras.models.Sequential([
    #tf.keras.layers.Flatten(input_shape=(512,1), name="simple_model"),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='tanh'),
    tf.keras.layers.Dense(200, activation='tanh'),
    tf.keras.layers.Dense(200, activation='tanh'),
    tf.keras.layers.Dense(1, name="predictions")
])


In [241]:
# train
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError(), metrics=['accuracy'])
model.fit(x=x,y=usol, epochs=50)


Epoch 1/50
16/16 [==============================] - 1s 5ms/step - loss: 0.0514 - accuracy: 0.0039
Epoch 2/50
16/16 [==============================] - 0s 4ms/step - loss: 0.0275 - accuracy: 0.0039
Epoch 3/50
16/16 [==============================] - 0s 5ms/step - loss: 0.0259 - accuracy: 0.0039
Epoch 4/50
16/16 [==============================] - 0s 5ms/step - loss: 0.0220 - accuracy: 0.0039
Epoch 5/50
16/16 [==============================] - 0s 4ms/step - loss: 0.0198 - accuracy: 0.0039
Epoch 6/50
16/16 [==============================] - 0s 5ms/step - loss: 0.0137 - accuracy: 0.0039
Epoch 7/50
16/16 [==============================] - 0s 5ms/step - loss: 0.0063 - accuracy: 0.0039
Epoch 8/50
16/16 [==============================] - 0s 5ms/step - loss: 8.2084e-04 - accuracy: 0.0039
Epoch 9/50
16/16 [==============================] - 0s 5ms/step - loss: 4.4587e-04 - accuracy: 0.0039
Epoch 10/50
16/16 [==============================] - 0s 5ms/step - loss: 2.6365e-04 - accuracy: 0.0039
Epoch 1

In [242]:
model.evaluate(x=x, y=usol)

16/16 [==============================] - 0s 4ms/step - loss: 6.4006e-06 - accuracy: 0.0039


[6.40064445178723e-06, 0.00390625]

In [250]:
# calculate gradient
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    forward = model(x)
    u_x = tape.gradient(forward, x, unconnected_gradients='zero')
    u_xx = tape.gradient(u_x, x, unconnected_gradients='zero')

#print(forward)
print(u_x)
print(u_xx)

tf.Tensor(
[[ 2.43167830e+00]
 [ 2.43249726e+00]
 [ 2.43330884e+00]
 [ 2.43411350e+00]
 [ 2.43491054e+00]
 [ 2.43570018e+00]
 [ 2.43648291e+00]
 [ 2.43725777e+00]
 [ 2.43802547e+00]
 [ 2.43878603e+00]
 [ 2.43953919e+00]
 [ 2.44028473e+00]
 [ 2.44102311e+00]
 [ 2.44175434e+00]
 [ 2.44247746e+00]
 [ 2.44319344e+00]
 [ 2.44390202e+00]
 [ 2.44460297e+00]
 [ 2.44529676e+00]
 [ 2.44598293e+00]
 [ 2.44666147e+00]
 [ 2.44733238e+00]
 [ 2.44799566e+00]
 [ 2.44865227e+00]
 [ 2.44930029e+00]
 [ 2.44994164e+00]
 [ 2.45057487e+00]
 [ 2.45123720e+00]
 [ 2.45185518e+00]
 [ 2.45246601e+00]
 [ 2.45306921e+00]
 [ 2.45366430e+00]
 [ 2.45425224e+00]
 [ 2.45483208e+00]
 [ 2.45540500e+00]
 [ 2.45596933e+00]
 [ 2.45652676e+00]
 [ 2.45707655e+00]
 [ 2.45761776e+00]
 [ 2.45815182e+00]
 [ 2.45867777e+00]
 [ 2.45919609e+00]
 [ 2.45970726e+00]
 [ 2.46021032e+00]
 [ 2.46070552e+00]
 [ 2.46119308e+00]
 [ 2.46167278e+00]
 [ 2.46214485e+00]
 [ 2.46260905e+00]
 [ 2.46306539e+00]
 [ 2.46351385e+00]
 [ 2.46395469e+00]
 